In [39]:
import numpy as np
import pandas as pd

In [86]:
file = 'global_data_4_1'
file_split = np.array(file.split('_'))[-2:].astype(float)

time_bin  = file_split[-1]
H_bin  = file_split[-2]

time_bins = np.arange(0,24,time_bin)
H_bins = np.arange(90,130,H_bin)

shape = len(time_bins) * len(H_bins)

file_path = f'../../Results/{file}.csv'
omni_path = '../../Index/omni_global.csv'

In [87]:
data_df = pd.read_csv(file_path)
omni_df = pd.read_csv(omni_path)
Date = np.array(data_df['Date']).reshape(-1,shape)[:,0]
Time = np.array(data_df['Hours']).reshape(-1,shape)[:,0]
Mask = np.array(data_df['EXP Density']).reshape(-1,shape) == 0.0

Date_index = omni_df['Date'].values
Time_index = omni_df['Time'].values
IMF = omni_df['IMF'].values
speed = omni_df['Speed'].values

if time_bin == 1:
    Date_index = Date_index.reshape(-1,2)[:,0]
    Time_index = Time_index.reshape(-1,2)[:,0]
    IMF = np.nanmean(IMF.reshape(-1,2), axis = 1)
    speed = np.nanmean(speed.reshape(-1,2), axis = 1)

IMF[np.isnan(IMF)] = 0
speed[np.isnan(speed)] = 0

next_IMF = np.concatenate(([IMF[-1]], IMF[0:-1]))
next_speed = np.concatenate(([speed[-1]], speed[0:-1]))

dIMF = (IMF-next_IMF)/time_bin
dspeed = (speed-next_speed)/time_bin

IMF_list = []
dIMF_list = []
speed_list = []
dspeed_list = []

for date in Date:
    date_index_mask = Date_index == date

    IMF_list.append(np.tile(IMF[date_index_mask], int(40/H_bin)))
    dIMF_list.append(np.tile(dIMF[date_index_mask], int(40/H_bin)))
    speed_list.append(np.tile(speed[date_index_mask], int(40/H_bin)))
    dspeed_list.append(np.tile(dspeed[date_index_mask], int(40/H_bin)))


IMF_array = np.array(IMF_list)
dIMF_array = np.array(dIMF_list)
speed_array = np.array(speed_list)
dspeed_array = np.array(dspeed_list)

IMF_array[Mask] = 0
dIMF_array[Mask] = 0
speed_array[Mask] = 0
dspeed_array[Mask] = 0

IMF_array = IMF_array.reshape(1, -1)[0]
dIMF_array = dIMF_array.reshape(1, -1)[0]
speed_array = speed_array.reshape(1, -1)[0]
dspeed_array = dspeed_array.reshape(1, -1)[0]

data_df['IMF'] = (IMF_array*100).astype(int)/100
data_df['IMF Gradient'] = (dIMF_array*100).astype(int)/100
data_df['Speed'] = (speed_array*100).astype(int)/100
data_df['Speed Gradient'] = (dspeed_array*100).astype(int)/100

data_df.to_csv(file_path, index=False)

/var/folders/np/43rq4vzs0js_3z1xt5mx73hr0000gn/T/ipykernel_54258/299634605.py:15: RuntimeWarning: Mean of empty slice
  IMF = np.nanmean(IMF.reshape(-1,2), axis = 1)
/var/folders/np/43rq4vzs0js_3z1xt5mx73hr0000gn/T/ipykernel_54258/299634605.py:16: RuntimeWarning: Mean of empty slice
  speed = np.nanmean(speed.reshape(-1,2), axis = 1)
